# Debug COEFF columns in specprod-db

In [1]:
import os
import numpy as np
from astropy.table import Table
from desiutil.log import get_logger
from desispec.io import findfile
import specprodDB.load as db
from sqlalchemy.sql.expression import func

In [2]:
# specprod = os.environ['SPECPROD'] = 'fuji'
# specprod = os.environ['SPECPROD'] = 'guadalupe'
specprod = os.environ['SPECPROD'] = 'iron'
# specprod = os.environ['SPECPROD'] = 'loa'
sample_size = 100  # Number of random samples to retrieve from the database.
sample_db = False  # If True, check the database, if False, check patch files.

In [3]:
db.log = get_logger()
postgresql = db.setup_db(schema=specprod, hostname='specprod-db.desi.lbl.gov', username='desi')


In [4]:
if sample_db:
    # Query on individual columns to allow this to run on fuji.
    random_zpix_query = db.dbSession.query(db.Zpix.targetid, db.Zpix.survey, db.Zpix.program, db.Zpix.healpix,
                                           db.Zpix.coeff_0, db.Zpix.coeff_1, db.Zpix.coeff_2, db.Zpix.coeff_3, db.Zpix.coeff_4,
                                           db.Zpix.coeff_5, db.Zpix.coeff_6, db.Zpix.coeff_7, db.Zpix.coeff_8, db.Zpix.coeff_9).order_by(func.random()).limit(sample_size).all()
else:
    zpix_patch_table = Table.read(os.path.join(os.environ['SCRATCH'], 'coeff_patch', f'zall-pix-{specprod}-coeff-patch.fits'), hdu='COEFF_PATCH')
    w = np.random.choice(len(zpix_patch_table), sample_size)
    random_zpix_query = zpix_patch_table[w]

In [5]:
random_zpix = dict()
for row in random_zpix_query:
    if sample_db:
        filename, exists = findfile('redrock', survey=row[1], faprogram=row[2], healpix=row[3], readonly=True, return_exists=True)
    else:
        filename, exists = findfile('redrock', survey=row['SURVEY'], faprogram=row['PROGRAM'], healpix=row['HEALPIX'], readonly=True, return_exists=True)
    # return_exists is acting weird for /dvs_ro and loa.
    if not exists:
        exists = os.path.exists(filename)
    if exists:
        if sample_db:
            key = (row[0], row[1], row[2])  # targetid, survey, program
            random_zpix[key] = {'row': row, 'filename': filename,
                                'db_coeff': np.array(row[4:])}
        else:
            key = (row['TARGETID'].tolist(), row['SURVEY'], row['PROGRAM'])
            random_zpix[key] = {'row': row, 'filename': filename,
                                'db_coeff': np.array([row[f'COEFF_{k:d}'].tolist() for k in range(10)])}
    else:
        print(f"WARNING: {filename} does not exist, skipping!")

In [6]:
if sample_db:
    random_ztile_subquery = db.dbSession.query(db.Fiberassign).order_by(func.random()).limit(sample_size).subquery()
    random_ztile_query = db.dbSession.query(db.Ztile.targetid, db.Ztile.tileid, db.Ztile.spgrpval, random_ztile_subquery.c.petal_loc,
                                            db.Ztile.coeff_0, db.Ztile.coeff_1, db.Ztile.coeff_2, db.Ztile.coeff_3, db.Ztile.coeff_4, db.Ztile.coeff_5,
                                            db.Ztile.coeff_6, db.Ztile.coeff_7, db.Ztile.coeff_8, db.Ztile.coeff_9).join(random_ztile_subquery,
                                                                                                                         (db.Ztile.targetid==random_ztile_subquery.c.targetid) & (db.Ztile.tileid==random_ztile_subquery.c.tileid)).all()
else:
    ztile_patch_table = Table.read(os.path.join(os.environ['SCRATCH'], 'coeff_patch', f'zall-tilecumulative-{specprod}-coeff-patch.fits'), hdu='COEFF_PATCH')
    w = np.random.choice(len(ztile_patch_table), sample_size)
    random_ztile_query = ztile_patch_table[w]

In [7]:
random_ztile = dict()
for row in random_ztile_query:
    if sample_db:
        filename, exists = findfile('redrock', groupname='cumulative', tile=row[1], night=row[2], spectrograph=row[3], readonly=True, return_exists=True)
    else:
        petal_query = db.dbSession.query(db.Fiberassign.petal_loc).filter(db.Fiberassign.targetid==row['TARGETID'].tolist()).filter(db.Fiberassign.tileid==row['TILEID'].tolist()).all()
        assert len(petal_query) == 1
        filename, exists = findfile('redrock', groupname='cumulative', tile=row['TILEID'], night=row['SPGRPVAL'], spectrograph=petal_query[0][0], readonly=True, return_exists=True)
    if not exists:
        exists = os.path.exists(filename)
    if exists:
        if sample_db:
            key = (row[0], row[1])  # targetid, tileid
            random_ztile[key] = {'row': row, 'filename': filename,
                                 'db_coeff': np.array(row[4:])}
        else:
            key = (row['TARGETID'].tolist(), row['TILEID'].tolist())  # targetid, tileid
            random_ztile[key] = {'row': row, 'filename': filename,
                                 'db_coeff': np.array([row[f'COEFF_{k:d}'].tolist() for k in range(10)])}
            
    else:
        print(f"WARNING: {filename} does not exist, skipping!")

In [8]:
if sample_db:
    random_photometry_query = db.dbSession.query(db.Photometry).order_by(func.random()).limit(sample_size).all()

In [9]:
if sample_db:
    random_photometry = dict()
    for row in random_photometry_query:
        if row.release == 9011:
            ns = 'north'
        else:
            ns = 'south'
        if row.brickname:
            ra = row.brickname[0:3]
            filename = f"/dvs_ro/cfs/cdirs/cosmo/data/legacysurvey/dr9/{ns}/tractor/{ra}/tractor-{row.brickname}.fits"
            exists = os.path.exists(filename)
            if exists:
                key = row.targetid
                random_photometry[key] = {'row': row, 'filename': filename}
                random_photometry[key]['db_dchisq'] = np.array([row.dchisq_psf, row.dchisq_rex, row.dchisq_dev, row.dchisq_exp, row.dchisq_ser])
            else:
                print(row)
                print(f"WARNING: {filename} does not exist, skipping!")
        else:
            print(f"INFO: Likely secondary target, {row}, skipping.")

In [10]:
randoms = {'zpix': random_zpix, 'ztile': random_ztile}
if sample_db:
    randoms['photometry']: random_photometry
hdus = {'zpix': 'REDSHIFTS', 'ztile': 'REDSHIFTS', 'photometry': 1}
for table in randoms:
    for key in randoms[table]:
        data = Table.read(randoms[table][key]['filename'], format='fits', hdu=hdus[table])
        if table == 'photometry':
            w = np.where(data['objid'] == randoms[table][key]['row'].brick_objid)[0]
        else:
            w = np.where(data['TARGETID'] == randoms[table][key]['row'][0])[0]
        if len(w) == 0:
            print(f"ERROR: object not found in {randoms[table][key]['filename']}!")
        elif len(w) > 1:
            print(f"ERROR: multiple objects found in {randoms[table][key]['filename']}!")
        else:
            if table == 'photometry':
                assert np.allclose(data[w]['dchisq'].data[0], randoms[table][key]['db_dchisq'])
            else:
                try:
                    assert np.allclose(data[w]['COEFF'].data[0], randoms[table][key]['db_coeff'])
                except AssertionError:
                    print(specprod, table, key, data[w]['COEFF'].data[0], randoms[table][key]['db_coeff'])